## Decorators

Decorators can be used to add functionality like logging, caching, timing, authorization, and more.

Coding exercise: decorators

We want you to build a decorator check_permission() that checks the user's role  (access level) and only allows 'admin ' to delete the database. If the user is not an admin, the decorator will raise a PermissionError, which is a built-in Python error, like the ones we have already seen. You may include an error message like You are not an admin. along with the PermissionError.

Then you will need to create a function secure_delete_database() using the check_permission() decorator and the original delete_database() function.

In [25]:
# ---- Do not change the code below ----
# User identity dictionary
user = {
    'id': 1,
    'name': 'jose',
    'role': 'admin'
}

# delete_database() function, DO NOT CHANGE
def delete_database():
    # perform deletion
    print('Database deleted!')

# ---- Do not change the code above ----


# You code starts here:
# Define a check_permission() decorator:
def check_permission(func):
    if user.get('role') == 'admin':
        return func()
    else:
        raise PermissionError("You don't have permission to delete this database.")

@check_permission
def secure_delete_database():
    return delete_database()

# Use the check_permission() decorator and delete_database() function to create a secure_delete_database() function

Database deleted!


In [22]:
# more flexible and reusable way to make the decorator

def check_permission(func):
    def wrapper():
        if user.get('role') == 'admin':
            return func()
        else:
            raise PermissionError('You are not an admin.')
    return wrapper

secure_delete_database = check_permission(delete_database)
secure_delete_database()

Database deleted!


Exercise: a generic access control decorator

When we are building a complicated system, it often involves many users with different roles (access levels). And for different roles, we may want to grant different privileges. For example, we may only want our admins to be able to delete some important files, while a regular user should not be able to modify them. This could be done without the help of a decorator indeed. However, there might be other operations that we would expect the same access control, and it could be repetitive to write the same piece of code that checks the user's access level in each of these functions.

In this exercise, we will be dealing exactly with this problem. We ask you to create an @access_control decorator that checks the user's access level and decides whether that user has the privilege to call the function being decorated.

Your decorator would work like this:

    @access_control(access_level)
    def delete_file(filename):
        # perform the deletion operation
        print(f'{filename} is deleted!')

In the above example, delete_file() is a very dangerous but essential function that should only work for our system admins. We would want to wrap it with the @access_control decorator and this decorator would take in an access_level argument that sets the bar to decide whether one can call the function or not.

Your decorator should meet the following requirements:

    It takes in an argument access_level and uses it to compare with the user's current access level .

    You can get the current user's role, represented by an integer, by calling the get_current_user_role() function. You don't need to implement this function, we will take care of it for you.

    You may assume smaller access level value would have higher privilege. For example, 0 - admin, 1 - user, 2 - guest. So you can check if the user has proper access level like this:

    if get_current_user_role() <= access_level:
        # do something
    else:
        # forbid

    If the user has the proper access level, we allow the user to call the function (that has this decorator).

    If the user does not have a proper access level, we raise a PermissionError with the message You do not have the proper access level.

    The decorator should be generic, which means it can be applied to any function that has any amount of arguments (or key word arguments).

    Your decorator should keep the original function's __name__ and __doc__ strings.

P.S: PermissionError is a Python built-in error, which works like the ones we've already seen, such as the ValueError.

In [26]:
"""
Implement a @access_control decorator that can be used like this:
@access_control(access_level)
def delete_some_file(filename):
    # perform the deletion operation
    print('{} is deleted!'.format(filename))
Your decorator should meet the following requirements:
- It takes in an argument `access_level` and would compare the current user's role with the access level.
- You can get the current user's role, represented by an integer, by calling the `get_current_user_role()` function.
    You don't need to implement this function, we will take care of it for you.
- You may assume smaller access level value would have higher privilege. For example, 0 - admin, 1 - user, 2 - guest.
    So you can check if the user has proper access level like this:
if get_current_user_role() <= access_level:
    # do something
else:
    # forbid
- If the user has the proper access level, we allow the user to call the function (that has this decorator).
- If the user does not have a proper access level, we raise a `PermissionError` with the message:
    'You do not have the proper access level.'
- The decorator should be generic, which means it can be applied to any function that has any amount of
    arguments (or key word arguments).
- Your decorator should keep the original function's `__name__` and `__doc__` strings.
"""
from functools import wraps

# DO NOT CHANGE
def get_current_user_role() -> int:
    # return the current user's role, represented by an int
    # for example, 0 - admin, 1 - user, 2 - guest
    # You don't need to change this function, we will replace it with a real function that returns the user's role
    return 0


def access_control(access_level: int):
    # You code starts here:
    def outer_wrapper(func):
        @wraps(func)
        def inner_wrapper(*arg, **kwargs):
            if get_current_user_role() <= access_level:
                return func(*arg, **kwargs)
            else:
                raise PermissionError('You do not have the proper access level.')
            return inner_wrapper
        return outer_wrapper

In [40]:
import functools

# Try the various combinations below!
user = {'username': 'jo', 'access_level': 'admin'}
# user = {'username': 'bob', 'access_level': 'admin'}
# user = {'username': 'jose123', 'access_level': 'user'}
#user = {'username': 'bob', 'access_level': 'user'}


def user_name_starts_with_j(func):
    """
    This decorator only runs the function passed if the user's username starts with a j.
    """
    @functools.wraps(func)
    def secure_func(*args, **kwargs):
        if user.get('username').startswith('j'):
            return func(*args, **kwargs)
        else:
            print("User's username did not start with 'j'.")
    return secure_func


def user_has_permission(func):
    """
    This decorator only runs the function passed if the user's access_level is admin.
    """
    @functools.wraps(func)
    def secure_func(*args, **kwargs):
        if user.get('access_level') == 'admin':
            return func(*args, **kwargs)
        else:
            print("User's access_level was not 'admin'.")
    return secure_func

@user_name_starts_with_j
@user_has_permission
def double_decorator():
    return 'I ran.'

print(double_decorator())

User's username did not start with 'j'.
None


In [1]:
def uppercase_decorator(function):
    def wrapper():
        result = function()
        return result.upper()
    return wrapper

@uppercase_decorator
def say_hello():
    return "hello"

print(say_hello())

HELLO


In [2]:
say_hello = uppercase_decorator(say_hello)

In [5]:
print(say_hello())

HELLO


In [7]:
def repeat(n):
    def decorator_function(function):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                result = function(*args, **kwargs)
            return result
        return wrapper
    return decorator_function

@repeat(3)
def greet(name):
    print(f"Hello, {name}!")

greet("Alex")

Hello, Alex!
Hello, Alex!
Hello, Alex!


In [9]:
def test_print(**kwargs):
    for k, v in kwargs.items():
        print(f'For {k} we have {v}.')
        
test_print(a='эй', b='би', c='си')

For a we have эй.
For b we have би.
For c we have си.


In [10]:
test_print(**{'a': 'эй', 'b': 'би', 'c': 'си'})

For a we have эй.
For b we have би.
For c we have си.


In [28]:
def make_bold(func):
    def wrapper():
        return "<b>" + func() + "</b>"
    return wrapper

def make_italic(func):
    def wrapper():
        return "<i>" + func() + "</i>"
    return wrapper

@make_bold
@make_italic
def say_hello():
    return "Hello, world!"

print(say_hello())

<b><i>Hello, world!</i></b>


In [43]:
def example_func(*args, **kwargs):
    for arg in args:
        print(arg)
    for key, value in kwargs.items():
        print(f"{key}: {value}")

example_func(1, 2, a=3, b=4)

1
2
a: 3
b: 4


In [44]:
example_func(1, 2)

1
2


In [45]:
example_func(a=3, b=4)

a: 3
b: 4
